<a href="https://colab.research.google.com/github/omerhac/arc_challenge/blob/master/board_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!pip install import_ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=96ac65d7ccaf0cb1bc21327beb03890c4c5e631948a85a43eafc8dcdcddcdc15
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [3]:
# get repository from github
!git clone https://github.com/omerhac/arc_challenge.git

# navigate to dir
%cd arc_challenge

Cloning into 'arc_challenge'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 188 (delta 3), reused 0 (delta 0), pack-reused 179
Receiving objects: 100% (188/188), 10.54 MiB | 10.13 MiB/s, done.
Resolving deltas: 100% (83/83), done.
/content/arc_challenge


In [4]:
import preprocess

importing Jupyter notebook from preprocess.ipynb


# Data loading and gathering


In [5]:
# load the lists of tasks
training_tasks, eval_tasks, test_tasks = preprocess.load_data_from_jsons()

In [6]:
# augment and arrange the data as an array
all_boards = preprocess.get_all_boards(training_tasks, eval_tasks, test_tasks)
all_boards += preprocess.get_all_boards(training_tasks, eval_tasks, test_tasks) # add another batch of randomly padd boards

# make all boards binary
all_boards = [preprocess.get_binary_board(board) for board in all_boards]

# get rotated views of all the boards
rotated_boards = []

for board in all_boards:
  rotated_views = preprocess.get_rotated_views(board)
  rotated_boards += rotated_views


In [7]:
print(len(rotated_boards))

136320


# GAN

In [11]:
## constants "" 
GENERATOR_DENSE_REP = 50
N_CHANNELS = 1

# Defining the generator

In [19]:
from tensorflow.keras.layers import Dense, Conv2DTranspose, BatchNormalization, Input
def generator():
  """
  Creates a board generator
  """

  def util_gen(z):
    """
    A utility generator function to be used in a lambda later on
    """

    # project and reshape z
    linear = Dense(128*4*4, activation='linear')(z)
    reshaped_z = tf.keras.layers.Reshape([4, 4, 64])(linear)

    # first deconv layer
    g_conv1 = Conv2DTranspose(64, kernel_size=(3,3), activation='relu')(reshaped_z)
    g_normalized_1 = BatchNormalization(momentum=0.9)(g_conv1)

    # second deconv layer
    g_conv2 = Conv2DTranspose(32, kernel_size=(3,3), activation='relu')(g_normalized_1)
    g_normalized_2 = BatchNormalization(momentum=0.9)(g_conv2)

    # third deconv layer
    g_conv3 = Conv2DTranspose(32, kernel_size=(3,3), activation='relu')(g_normalized_2)
    g_normalized_3 = BatchNormalization(momentum=0.9)(g_conv3)

    # fourth deconv layer
    g_conv4 = Conv2DTranspose(16, kernel_size=(3,3), activation='relu')(g_normalized_3)
    g_normalized_4 = BatchNormalization(momentum=0.9)(g_conv4)

    # fifth deconv layer
    g_conv5 = Conv2DTranspose(8, kernel_size=(3,3), activation='relu')(g_normalized_4)
    g_normalized_5 = BatchNormalization(momentum=0.9)(g_conv5)

    # final layer
    output_board = Conv2DTranspose(1, kernel_size=(3,3), activation='relu')(g_normalized_5)

    return output_board

  return lambda z: util_gen(z)


## Defining discriminator 

In [27]:
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D
def discriminator():
  """
  Creates a discriminator
  """

  def util_discriminator(board):
    """
    A utility discriminator to be used in a lambda later on
    """

    # first layer
    d_conv1 = Conv2D(8, kernel_size=(3,3), activation='relu', name="d_conv1")(board)
    d_normalized_1 = BatchNormalization(momentum=0.9)(d_conv1)

    # second layer
    d_conv2 = Conv2D(16, kernel_size=(3,3), activation='relu', name="d_conv2")(d_normalized_1)
    d_normalized_2 = BatchNormalization(momentum=0.9)(d_conv2)

    # third layer
    d_conv3 = Conv2D(32, kernel_size=(3,3), activation='relu', name="d_conv3")(d_normalized_2)
    d_normalized_3 = BatchNormalization(momentum=0.9)(d_conv3)

    # fourth layer
    d_conv4 = Conv2D(64, kernel_size=(3,3), activation='relu', name="d_conv4")(d_normalized_3)
    d_normalized_4 = BatchNormalization(momentum=0.9)(d_conv4)

    # fifth layer
    d_conv5 = Conv2D(128, kernel_size=(3,3), activation='relu', name="d_conv5")(d_normalized_4)
    d_normalized_5 = BatchNormalization(momentum=0.9)(d_conv5)

    # avarage_pooling
    d_ap = GlobalAveragePooling2D()(d_normalized_5)

    # dense
    d_out = Dense(1, activation='sigmoid')(d_ap)

    return d_out
  
  return lambda board: util_discriminator(board)

In [28]:
def get_model():
  z = Input([16,16,1], name='z')
  output = discriminator()(z)
  
  model = tf.keras.Model(inputs=z, outputs=output)
  model.compile()
  return model

m = get_model()
m.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               [(None, 16, 16, 1)]       0         
_________________________________________________________________
d_conv1 (Conv2D)             (None, 14, 14, 8)         80        
_________________________________________________________________
batch_normalization_13 (Batc (None, 14, 14, 8)         32        
_________________________________________________________________
d_conv2 (Conv2D)             (None, 12, 12, 16)        1168      
_________________________________________________________________
batch_normalization_14 (Batc (None, 12, 12, 16)        64        
_________________________________________________________________
d_conv3 (Conv2D)             (None, 10, 10, 32)        4640      
_________________________________________________________________
batch_normalization_15 (Batc (None, 10, 10, 32)        128 